In [30]:
from sklearn.decomposition import KernelPCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np
import xlwt

import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
from ArcCosineKernel import ArcCosineKernel0,ArcCosineKernel1,ArcCosineKernel2,ArcCosineKernel
from sklearn.feature_selection import VarianceThreshold,SelectKBest,mutual_info_classif
from pylmnn import LargeMarginNearestNeighbor as LMNN

In [20]:
data = pd.read_csv('mnist_test.csv',header = None).iloc[5000:,:].sample(n=3000)
data_X = data.iloc[:,1:].to_numpy()
data_y = data.iloc[:,0].to_numpy()
data_test = pd.read_csv('mnist_test.csv',header = None).iloc[:5000,:].sample(n=1000)
data_test_X = data_test.iloc[:,1:].to_numpy()
data_test_y = data_test.iloc[:,0].to_numpy()

In [19]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter classifier: classifier used for the output layer
# parameter arc_order: the order of activation function and arccosine kernel
# return the trained MKM model

# classified by KNN with cv

def MKM(X,y,l,kernel,arc_order,layer = 1):
    New_X,New_y = X,y
    print(New_X.shape)
    train_list = []
    KPCA_list = []
    FS_list = []
    w_list = []
    for i in range(l):
        size = New_X.shape[1]
        # K-layer
        train_list.append(New_X)
        gram_train = ArcCosineKernel(New_X,New_X,arc_order)
        KPCA = KernelPCA(n_components = size-80, kernel = kernel)
        K_new_X,K_new_y = KPCA.fit_transform(gram_train),New_y
        # Bookkeeping the KPCA model used in each layer
        KPCA_list.append(KPCA)
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,4),range(10,30,5))
#         w_list.append(w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-100)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,200)
#         FS_list.append(dic_kept)
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Train Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    lmnn = LMNN(n_neighbors=50, max_iter=100, n_components= final_X.shape[1])
    lmnn.fit(final_X, final_y)
    knn = KNN(n_neighbors = 10)
    model = knn.fit(lmnn.transform(final_X), final_y)
    print('Model has been successfully trained!')
    print('----------------------------------------------------------------------')
    return model,KPCA_list,FS_list,w_list,train_list

In [21]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter data_test_X: data array of shape(n_sample,n_feature)
# parameter data_test_y: labels of shape(n_samples,)
# parameter l: the number of layers of kernel machines
# parameter MKM: the trained MKM model
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter arc_order: the order of activation function and arccosine kernel
# return accuracy score for test set

# classified by KNN with cv

def MKM__test(X,y,l,arc_order,KPCA_list,FS_list,w_list,train_list,MKM):
    New_X,New_y = X,y
    for i in range(l):
        size = New_X.shape[1]
        dic_feature = {}
        for f in range(size):
            dic_feature[f] = New_X[:,f]
        # K-layer
        gram_test = ArcCosineKernel(New_X,train_list[i],arc_order)
        K_new_X,K_new_y = KPCA_list[i].transform(gram_test),New_y
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,15),range(10,300))
#         P_new_X,P_new_y = mutual_info_FS(K_new_X,K_new_y,w)
#         P_new_X,P_new_y = mutual_info_FS(K_new_X,K_new_y,209)
#         for key in FS_list[i].keys():
#             FS_list[i][key] = dic_feature[key]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-80-w_list[i]]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-100]
#         P_new_X,P_new_y = temp,K_new_y
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Test Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    predict_y = MKM.predict(final_X)
    print(accuracy_score(final_y,predict_y))
    print('Test is finished!')
    print('----------------------------------------------------------------------')
    return accuracy_score(final_y,predict_y)

In [22]:
model, KPCA_list ,FS_list,w_list,train_list= MKM(data_X,data_y,1,'precomputed',0)

(3000, 784)
(3000, 3000)
Train Layer 1 Done!
Data has the shape (3000, 704)
Model has been successfully trained!
----------------------------------------------------------------------


In [23]:
MKM__test(data_test_X,data_test_y,1,0,KPCA_list,FS_list,w_list,train_list,model)

(1000, 3000)
Test Layer 1 Done!
Data has the shape (1000, 704)
0.604
Test is finished!
----------------------------------------------------------------------


0.604

In [39]:
accu_score_temp = []
for d in range(5):
    # load and sample the training and test dataset
    data = pd.read_csv('mnist_test.csv',header = None).iloc[:5000,:].sample(n=3000)
    data_X = data.iloc[:,1:].to_numpy()
    data_y = data.iloc[:,0].to_numpy()
    data_test = pd.read_csv('mnist_test.csv',header = None).iloc[5000:,:].sample(n=1000)
    data_test_X = data_test.iloc[:,1:].to_numpy()
    data_test_y = data_test.iloc[:,0].to_numpy()

    model, KPCA_list ,FS_list,w_list,train_list= MKM(data_X,data_y,1,'precomputed',2)
    accu_score_temp.append(MKM__test(data_test_X,data_test_y,1,2,KPCA_list,FS_list,w_list,train_list,model))

(3000, 784)
(3000, 3000)
Train Layer 1 Done!
Data has the shape (3000, 704)
Model has been successfully trained!
----------------------------------------------------------------------
(1000, 3000)
Test Layer 1 Done!
Data has the shape (1000, 704)
0.561
Test is finished!
----------------------------------------------------------------------
(3000, 784)
(3000, 3000)
Train Layer 1 Done!
Data has the shape (3000, 704)
Model has been successfully trained!
----------------------------------------------------------------------
(1000, 3000)
Test Layer 1 Done!
Data has the shape (1000, 704)
0.182
Test is finished!
----------------------------------------------------------------------
(3000, 784)
(3000, 3000)
Train Layer 1 Done!
Data has the shape (3000, 704)
Model has been successfully trained!
----------------------------------------------------------------------
(1000, 3000)
Test Layer 1 Done!
Data has the shape (1000, 704)
0.404
Test is finished!
---------------------------------------------

In [ ]:
accu_score_1 = []
# number of layers
for i in range(6):
    print('layer({})'.format(i))
    # datasets test to ensure robust
    accu_score_temp = []
    for d in range(5):
        # load and sample the training and test dataset
        data = pd.read_csv('mnist_test.csv',header = None).iloc[:5000,:].sample(n=3000)
        data_X = data.iloc[:,1:].to_numpy()
        data_y = data.iloc[:,0].to_numpy()
        data_test = pd.read_csv('mnist_test.csv',header = None).iloc[5000:,:].sample(n=1000)
        data_test_X = data_test.iloc[:,1:].to_numpy()
        data_test_y = data_test.iloc[:,0].to_numpy()
        
        model, KPCA_list ,FS_list,w_list,train_list= MKM(data_X,data_y,1,'precomputed',2)
        accu_score_temp.append(MKM__test(data_test_X,data_test_y,i,1,KPCA_list,FS_list,w_list,train_list,model))
        
        model, KPCA_list ,FS_list,w_list = MKM(data_X,data_y,i,kernel = 'linear')
        accu_score_temp.append(MKM__test(data_test_X,data_test_y,i,
                                        KPCA_list = KPCA_list,FS_list = FS_list,
                                        w_list = w_list,MKM = model))
    accu_score_1.append(np.round(sum(accu_score_temp)/len(accu_score_temp),3))
    print('----------------------------------------------------------------')

In [40]:
print(accu_score_temp)

[0.561, 0.182, 0.404, 0.246, 0.495]


In [41]:
workbook = xlwt.Workbook()  # 新建一个工作簿
sheet = workbook.add_sheet('KPCA(arccosine2)+LMNN')
for i in range(len(accu_score_temp)):
    sheet.write(0,i,accu_score_temp[i])
workbook.save('KPCA(arccosine2)+LMNN-layer1.xls') 